In [20]:
import re
import numpy as np
import pandas as pd
import pickle
from eunjeon import Mecab
from gensim.models import Word2Vec as w2v
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [21]:
#21대 국회 법안 데이터 전처리

df = pd.read_excel('21th_history.xlsx')

In [22]:
#가결/부결 코딩

for i in range(len(df)):
    if df.loc[i, '처리구분'] == 1 or df.loc[i, '처리구분'] == 2 or df.loc[i, '처리구분'] == 3 or df.loc[i, '처리구분'] == 4:
        df.loc[i, '처리구분'] = 1
    elif df.loc[i, '처리구분'] == 5 or df.loc[i, '처리구분'] == 9:
        df.loc[i, '처리구분'] = 1
    elif df.loc[i, '처리구분'] == 6 or df.loc[i, '처리구분'] == 7 or df.loc[i, '처리구분'] == 8:
        df.loc[i, '처리구분'] = 0
    elif df.loc[i, '처리구분'] == 10:
        df.loc[i, '처리구분'] = 0

In [23]:
#소관위 원-핫코딩
committee_set = list(set(df['소관위']))
committee = []
for com in committee_set:
    coms = com.split(", ")
    for c in coms:
        committee.append(c)
        
committee = list(set(committee))

for i in range(len(df)):
    if df['소관위'][i].__contains__('특별위원회'):
            df.loc[i, '특별위원회'] = 1
    for com in committee:
        if df['소관위'][i].__contains__(com):
            df.loc[i, com] = 1
        else:
            df.loc[i, com] = 0

In [24]:
for i in range(len(df)):
    if df.loc[i, '소속정당 구분'] == '더불어민주당' or df.loc[i, '소속정당 구분'] == '열린민주당':
        df.loc[i, 'party'] = 0
    elif df.loc[i, '소속정당 구분'] == '국민의힘' or df.loc[i, '소속정당 구분'] == '미래통합당' :
        df.loc[i, 'party'] = 1
    elif df.loc[i, '소속정당 구분'] == '국민의당' or df.loc[i, '소속정당 구분'] == '기본소득당' or df.loc[i, '소속정당 구분'] == '시대전환' or df.loc[i, '소속정당 구분'] == '정의당':
        df.loc[i, 'party'] = 2
    elif df.loc[i, '제안자'] == '위원장':
        df.loc[i, 'party'] = 4
    elif df.loc[i, '제안자'] == '정부':
        df.loc[i, 'party'] = 5
    else:
        df.loc[i, 'party'] = 3

In [25]:
for i in range(len(df)):
    if df.loc[i, '소속정당 구분'] == '더불어민주당' or df.loc[i, '소속정당 구분'] == '열린민주당' :
        df.loc[i, 'num_seats'] = 185
    elif df.loc[i, '소속정당 구분'] == '국민의힘' :
        df.loc[i, 'num_seats'] = 103
    elif df.loc[i, '소속정당 구분'] == '미래통합당' :
        df.loc[i, 'num_seats'] = 84
    elif df.loc[i, '소속정당 구분'] == '정의당' or df.loc[i, '소속정당 구분'] == '기본소득당' or df.loc[i, '소속정당 구분'] == '시대전환':
        df.loc[i, 'num_seats'] = 185
    elif df.loc[i, '소속정당 구분'] == '국민의당':
        df.loc[i, 'num_seats'] = 3
    elif df.loc[i, '소속정당 구분'] == '무소속':
        df.loc[i, 'num_seats'] = 9

In [26]:
#법안별 주요내용이 26개 카테고리 주제와 얼마나 유사한지 유사도 계산

model = w2v.load('word2vec.model') #모델 불러오기

tagger = Mecab()

summary_re=[]
for text in df['제안내용']:
    try:
        summary_re.append(re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text))
    except:
        summary_re.append('NaN')
    
cnt_words = []
summary_re3 = []

for text in summary_re:
    pos_text = tagger.pos(text)
    word_list = []
    for word in pos_text:
        word_list.append(word[0])
    cnt_words.append(len(word_list)+1)
    summary_re3.append(' '.join(word_list))
    
df['제안내용_re']=summary_re3

In [27]:
words = list()
for word in model.wv.vocab:
    words.append(word)

#주제 카테고리 dic
category = {'국회': ['의회', '국회'],
            '정당/선거': ['정당', '선거'] ,
            '안보': ['군사', '안보', '북한'], 
            '사법': ['사법', '법원', '검찰', '소송'],
            '행정' : ['행정', '지자체', '자치', '경찰', '공무원'],
            '재정' : ['재정', '예산', '회계'], 
            '중소기업' : ['중소기업', '창업', '벤처', '스타트업'],
            '에너지' : ['에너지', '수소', '전기', '가스'], 
            '부동산' : ['주택', '부동산', '주거', '임대차'], 
            '금융' : ['금융', '투자', '자본', '보험업'], 
            '자동차' : ['자동차', '승합자동차', '이륜자동차', '승용차'],
            '건설/기계/조선' : ['기계', '건설업', '건설사', '건축', '조선업', '해운업', '발주'],
            '유통/무역' : ['유통', '물류', '무역', '쇼핑몰', '마트', '백화점'], 
            'IT' : ['IT', '통신', '게임', '데이터', '인공지능', '블록체인', '클라우드'], 
            '농축산' : ['농업', '축산', '수산'],
            '복지':['복지', '연금', '빈곤', '수당'],
            '의료/보건' : ['의료', '보건', '병원', '질병', '의약품'] , 
            '도시/교통':['도시', '교통', '운전', '도로'],
            '교육' : ['교육', '학교', '대학', '유치원', '입시'], 
            '환경' : ['친환경', '오염', '저탄소', '온실가스', '기후', '수자원'], 
            '노동' : ['노동', '임금', '노동조합', '퇴직', '채용', '근로'], 
            '치안/안전' : ['치안', '범죄', '안전', '사고', '소방', '형사'],  
            '가족' : ['가족', '아동', '청소년'],
            '여성' : ['여성', '출산', '육아', '성범죄', '성희롱'], 
            '예체능' : ['예술', '영화', '음악', '전시','공연', '문화재', '체육', '방송', '언론', '스포츠', '콘텐츠']
            }


for key in category.keys():
    keyword_similar = np.zeros(shape=(len(words), ))
    
    for keyword in category[key]:
        keyword_similar_2 = []
        for word in words:
            sm = model.wv.similarity(keyword, word)
            if sm >= 0.8:
                keyword_similar_2.append(sm)
            else:
                keyword_similar_2.append(0)

        keyword_similar_2 = np.asarray(keyword_similar_2)
        keyword_similar += keyword_similar_2
    
    keyword_similar = list(keyword_similar)
    keyword_similar_df = pd.DataFrame({'words': words, 'similar' : keyword_similar})
    keyword_similar_df = keyword_similar_df.sort_values(by=['words'], axis=0)
    keyword_similar_df = keyword_similar_df.set_index('words')
    keyword_similar_df = keyword_similar_df.T

    vectorizer = CountVectorizer(vocabulary = keyword_similar_df.columns, binary=False)
    X = vectorizer.fit_transform(summary_re3)
    words_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
    words_df = words_df.T

    tdm = np.dot(keyword_similar_df, words_df)
    tdm = tdm.T
    tdm = np.ravel(tdm) #2차원 배열을 1차원으로 평평하게 만듬
    df[key] = list(tdm / cnt_words)

In [28]:
#공동발의데이터 전처리
spon = pd.read_csv('21th_sponsorship_0104.csv')
spon = spon.loc[ spon['의안번호'].isin(df['법안코드']) ].reset_index().drop(['index'],axis=1)
spon = spon[spon['구분']!='대표'].fillna(0)
spon = spon[spon['당선횟수']!=0]

In [29]:
#공동발의 의원들의 평균 선수
avg_elected = spon.pivot_table(index=['의안번호'], values=['당선횟수'], aggfunc=np.mean)
avg_elected = avg_elected.reset_index()

In [30]:
#정당다양성 변수: 공동발의 의원이 모두 같은 당 소속이면 0, 아니면 1

party_set = []

bill_no = list(set(spon['의안번호']))

for no in bill_no:
    bill = spon[spon['의안번호']==no]
    parties = set(bill['정당'])
    if len(parties) >= 2:
        party_set.append(1)
    else:
        party_set.append(0)
        
diversity = pd.DataFrame({'의안번호': bill_no, '정당다양성':party_set})

In [31]:
extra_var = pd.merge(avg_elected, diversity, how='left', on='의안번호')
extra_var = extra_var.rename(columns={"당선횟수": "공동발의평균선수"})
extra_var = extra_var.rename(columns={"의안번호": "법안코드"})

df = pd.merge(df, extra_var, how='left', on='법안코드')

In [32]:
#상임위 상정 여부 변수

for i in range(len(df)):
    if type(df.loc[i, '위원회 상정일']) == pd._libs.tslibs.timestamps.Timestamp:
        df.loc[i, '상임위 상정 여부'] = 1
    else:
        df.loc[i, '상임위 상정 여부'] = 0

# 반대당발언횟수 추가

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("bills.db", isolation_level=None)
cursor = conn.cursor()

cursor.execute("select * from bills_2021 WHERE proposeDt >= '2020-06-01' and passGubn = '계류의안'")

rows = cursor.fetchall() 
cols = [column[0] for column in cursor.description]

df = pd.DataFrame.from_records(data=rows, columns=cols)

In [3]:
df.columns

Index(['billId', 'billName', 'billNo', 'passGubn', 'procStageCd', 'proposeDt',
       'proposerKind', 'summary', 'generalResult', 'procDt', 'index', '공동발의자수',
       '공동발의평균선수', 'diversity', 'presentDt', 'COMMITTEE', 'PROC_RESULT',
       'RST_PROPOSER', 'PUBL_PROPOSER', 'COMMITTEE_ID', 'polyNm', '당선횟수',
       '선출형태'],
      dtype='object')

In [16]:
for i in range(len(df)):
    if df.loc[i, 'presentDt']==None or df.loc[i, 'presentDt']=='None':
        print(df.loc[i, 'billId'] + ": " + df.loc[i, 'billName'])

PRC_L2R0K0Y7I3I1M1I7S1Y7Z0W9Z0I0D6: 주한미군 공여구역주변지역 등 지원 특별법 일부개정법률안(곽상도의원 등 12인)
PRC_B2O0N0S7C3Q1N1W6E4P8R2N9I2Q3C3: 2023 세계스카우트잼버리 지원 특별위원회 구성결의안(안규백의원 등 32인)
PRC_Y2J0U0T7A3U1X1G4L5O2T5V5E8L1G9: 경찰법 일부개정법률안(정청래의원 등 12인)
PRC_Z2U0S0B7T3I0E1A5A3J2F0H5X3Q4L2: 도로교통법 일부개정법률안(이명수의원 등 15인)
PRC_U2Z0K0H7G3K0W1R4X5O9L0S7O9B8H1: 주택도시기금법 일부개정법률안(김은혜의원 등 11인)
PRC_H2J0Z0B7M3X0S1M4G5R7B1F6C8P5H3: 공공주택 특별법 일부개정법률안(김은혜의원 등 14인)
PRC_T2A0H0N7I2C8N1C8H3W3E4D5V0C0R6: 국가기후환경회의 위원 추천의 건(대통령)
PRC_S2X0Y0J7B2N8D1R3O4G2F3X8T2C9M2: 공직선거법 일부개정법률안(소병철의원 등 12인)
PRC_Y2M0X0H7Z2P8L1L1Y0T9H0Y1D8T9O2: 지방교부세법 일부개정법률안(이상민의원 등 10인)
PRC_J2E0Q0M7X2Z7P1B7M5Z7Y4D1O5M6W7: 아동학대범죄의 처벌 등에 관한 특례법 일부개정법률안(신동근의원 등 10인)
PRC_L2M0S0T7I2W7P1K6Z2G3R3Y3N2Y9Z0: 도로교통법 일부개정법률안(조승래의원 등 12인)
PRC_C2B0V0D7G2O4X1T0G4U6J1O5L9J6S6: 한국신문윤리위원회 윤리위원 추천의건(한국신문윤리위원회 이사장)
PRC_D2C0Y0H7H2B4W1S7Y3D9K1A6B8R0L2: 집회 및 시위에 관한 법률 일부개정법률안(전용기의원 등 10인)
PRC_V2R0Y0G7S2Z4S1G4B3B4G5R8H5Q9D6: 화재예방, 소방시설 설치·유지 및 안전관리에 관한 법률 일부개정법률안(박성중의원 등 11인)
PRC_F2Z0D0I7Q2F4H1F3N4B8P4R9

In [71]:
df[df['billId']=='PRC_B2T0C0I9W0W3S1X1F0G4E3X2R1M1Y4']['presentDt']

109    None
Name: presentDt, dtype: object

In [7]:
idx = 'PRC_Q2T1M0X1D0M4W1T4M3O0R3Y4C7O3D2'
url = f"http://apis.data.go.kr/9710000/BillInfoService2/getBillCommissionExaminationInfo?bill_id={idx}&ServiceKey=" + MYKEY2
req = requests.get(url)
xpars = xmltodict.parse(req.text)
jsonDump = json.dumps(xpars)
jsonBody = json.loads(jsonDump)
dt = jsonBody['response']['body']['JurisdictionExamination']['item']
dt

{'committeeName': '문화체육관광위원회',
 'presentDt': None,
 'procDt': None,
 'procResultCd': None,
 'submitDt': '2021-01-18'}

In [10]:
conn = sqlite3.connect("bills.db", isolation_level=None)
cursor = conn.cursor()

cursor.execute("select * from bills_2021")
rows = cursor.fetchall()
cols = [column[0] for column in cursor.description]

df = pd.DataFrame.from_records(data=rows, columns=cols)

In [40]:
df.columns

Index(['billId', 'billName', 'billNo', 'passGubn', 'procStageCd', 'proposeDt',
       'proposerKind', 'summary', 'generalResult', 'procDt', 'index', '공동발의자수',
       '공동발의평균선수', 'diversity', 'presentDt', 'COMMITTEE', 'PROC_RESULT',
       'RST_PROPOSER', 'PUBL_PROPOSER', 'COMMITTEE_ID', 'polyNm', '당선횟수',
       '선출형태'],
      dtype='object')

In [49]:
mykey = "f9191bcb5fc3472890a5e84347ae5ebb"
url3 = f"https://open.assembly.go.kr/portal/openapi/nwvrqwxyaytdsfvhu?KEY={mykey}&Type=json&pSize=300"
req = requests.get(url3).json()
congressman = pd.DataFrame(req['nwvrqwxyaytdsfvhu'][1]['row'])

for i in range(len(congressman)):
    congressman.loc[i, '당선횟수'] = len(congressman.loc[i, 'UNITS'].split(', '))

In [69]:
congressman[congressman['HG_NM']=='양금희']

,HG_NM,HJ_NM,ENG_NM,BTH_GBN_NM,BTH_DATE,JOB_RES_NM,POLY_NM,ORIG_NM,ELECT_GBN_NM,CMIT_NM,...,SEX_GBN_NM,TEL_NO,E_MAIL,HOMEPAGE,STAFF,SECRETARY,SECRETARY2,MONA_CD,MEM_TITLE,당선횟수
145,양금희,梁琴喜,YANG KUMHEE,양,1961-11-15,위원,국민의힘,대구 북구갑,지역구,산업통상자원중소벤처기업위원회,...,여,02-784-7451~7453,ygoldh8999@naver.com,https://blog.naver.com/ygoldh8999,"박종욱, 이현진","김홍석, 이은정","유도영, 김균서, 양지, 권기환, 백승미",VG24001G,- 학 력 \r\n1968.03 ~ 1974.02 대구 남산초등학교\r\n1974....,1.0


In [55]:
congressman_re = congressman[['HJ_NM', 'POLY_NM', '당선횟수', 'ELECT_GBN_NM']]
congressman_re[congressman_re['HJ_NM']=='成一鍾']

,HJ_NM,POLY_NM,당선횟수,ELECT_GBN_NM
124,成一鍾,국민의힘,2.0,지역구


In [6]:
import requests
import xmltodict
import json
import pandas as pd


billId = 'PRC_Q2T1M0X1D0M4W1T4M3O0R3Y4C7O3D2'
MYKEY2 = "sh1BLNic10zE0pynUHLuP0%2FDxTd5Fi4m5%2B4CojHK%2B%2BXTxH9ykyO3yVPROWHp3zsR9%2BB38%2BkIGmWgHB%2BYfmUB6A%3D%3D"
url2 = f"http://apis.data.go.kr/9710000/BillInfoService2/getBillPetitionMemberList?gbn1=bill&gbn2=reception&bill_id={billId}&ServiceKey=" + MYKEY2

req = requests.get(url2)
xpars = xmltodict.parse(req.text)

jsonDump = json.dumps(xpars)
jsonBody = json.loads(jsonDump)

date = pd.DataFrame(jsonBody['response']['body']['items']['item'])

In [3]:
for i in range(len(date)):
    if date.loc[i, 'polyNm'] == '미래통합당':
        date.loc[i, 'polyNm'] = '국민의힘'

In [5]:
date.loc[0, 'polyNm']

'더불어민주당'

In [15]:
a = df[df['선출형태']!='비례대표']
b = a[a['선출형태']!='지역구']
c = b[b['proposerKind']=='의원']
set(c['RST_PROPOSER'])

{None, '박덕흠', '전봉민'}

In [20]:
c

,billId,billName,billNo,passGubn,procStageCd,proposeDt,proposerKind,summary,generalResult,procDt,...,diversity,presentDt,COMMITTEE,PROC_RESULT,RST_PROPOSER,PUBL_PROPOSER,COMMITTEE_ID,polyNm,당선횟수,선출형태
7,PRC_F2Q0P0H6O3Z0W1X8T1J4B5V4L4C3W0,“6.25전쟁 납북희생자 기억의 날” 국가기념일 지정 촉구 결의안(태영호의원 등 20인),2101232,계류의안,소관위접수,2020-06-30,의원,None,None,None,...,1.0,None,None,None,None,None,NaN,None,NaN,None
41,PRC_M2P0I0T6D3P0P1Q4S2M4K4I6F7V9O6,기후위기 비상 대응 촉구 결의안(한정애의원 등 48인),2101198,처리의안,대안반영폐기,2020-06-30,의원,None,대안반영폐기,2020-09-24,...,1.0,2020-09-15,None,None,None,None,NaN,None,NaN,None
112,PRC_M2N0E0R6Q2D9V1E3Q5S3Y3B1P1L5Y2,해운항만산업 경쟁력 강화를 위한 정책지원 촉구 결의안(윤재갑의원 등 10인),2101127,계류의안,소관위심사,2020-06-29,의원,None,None,None,...,1.0,2020-09-16,None,None,None,None,NaN,None,NaN,None
182,PRC_A2I0N0V6Y2N6N1Y5H4K7O5X1J7Q4Y8,6·25전쟁 70주년 한미동맹의 지속적인 발전을 위한 특별 결의안(김성원의원 등 11인),2101058,처리의안,대안반영폐기,2020-06-26,의원,None,대안반영폐기,2020-12-09,...,0.0,2020-09-28,None,None,None,None,NaN,None,NaN,None
218,PRC_L2X0R0H6X2W5Q1V7C5F5M2F2X2I0W9,"코로나19 극복을 위해 헌신한 의료인, 자원봉사자, 일선 방역 공무원에 대한 감사 ...",2101024,계류의안,소관위접수,2020-06-25,의원,None,None,None,...,1.0,None,None,None,None,None,NaN,None,NaN,None
224,PRC_C2F0D0T6P2H5J1Y7N0W5T5M5U3B9R1,고위공직자범죄수사처장후보추천위원회의 운영 등에 관한 규칙안(유상범의원 등 43인),2101018,계류의안,소관위심사,2020-06-25,의원,제안이유 \r\n\r\n 고위공직자범죄수사처를 설치하고 고위공직자범죄수사처장 후보...,None,None,...,0.0,2020-09-24,None,None,None,None,NaN,None,NaN,None
255,PRC_Q2S0W0E6E2R5C1A4Y1I6M2J2P0S8J7,탈원전 정책 조정·피해보상 논의 및 공론화를 위한 특별위원회 구성결의안(강기윤의원 ...,2100987,계류의안,소관위접수,2020-06-25,의원,None,None,None,...,0.0,None,None,None,None,None,NaN,None,NaN,None
341,PRC_I2Z0I0S6B2R3S1V7U0L4V2N0Z5G6T1,북한의 대남 도발 규탄 및 북핵 폐기 촉구 결의안(조태용의원 등 103인),2100901,계류의안,소관위심사,2020-06-23,의원,None,None,None,...,0.0,2020-09-28,None,None,None,None,NaN,None,NaN,None
354,PRC_R2G0N0V6V2I3X1Z4W3G6I0A1D1O8J4,일본의 하시마(군함도) 탄광 등 한국인의 본인 의사에 반한 강제노동 동원에 대한 U...,2100888,처리의안,본회의의결,2020-06-23,의원,None,원안가결,2020-12-09,...,1.0,2020-09-28,None,None,None,None,NaN,None,NaN,None
371,PRC_S2G0H0C6T2K3P1T1R1R0R0L8S2A5E5,6·25전쟁 70주년 기념 호국영령과 국내외 참전용사에 대한 감사(感謝) 결의안(이...,2100871,계류의안,소관위심사,2020-06-23,의원,None,None,None,...,1.0,2020-07-28,None,None,None,None,NaN,None,NaN,None


In [20]:
for i in range(len(df)):
    a = df.loc[i, 'party']
    if i <= 10:
        print(float(a))
    df.loc[i, 'party'] = float(a)

0.0
5.0
0.0
0.0
0.0
0.0
5.0
1.0
1.0
0.0
1.0


In [21]:
df.loc[10, 'party']

1

In [22]:
set(df['party'])

{0, 1, 2, 3, 4, 5}

In [16]:
a = df.loc[10, 'party']
float(a)

1.0

In [23]:
committee = ['보건복지위원회',
       '행정안전위원회', '산업통상자원중소벤처기업위원회', '문화체육관광위원회', '국회운영위원회', '법제사법위원회',
       '국방위원회', '국토교통위원회', '기획재정위원회', '특별위원회', '여성가족위원회', '농림축산식품해양수산위원회',
       '외교통일위원회', '정보위원회', '과학기술정보방송통신위원회', '환경노동위원회', '교육위원회', '정무위원회']

for com in committee:
    if com not in df.columns:
        df[com] = 0
        
data = pd.get_dummies(df,columns=['입법형태','선출형태', 'party', '정당다양성', '상임위 상정 여부'])

In [24]:
data.columns

Index(['법안코드', '법안명', 'billNo', 'passGubn', 'procStageCd', '접수일자',
       'proposerKind', 'summary', 'generalResult', 'procDt', 'index', '공동발의자수',
       '공동발의평균선수', 'presentDt', 'COMMITTEE', 'PROC_RESULT', '제안자',
       'PUBL_PROPOSER', 'COMMITTEE_ID', 'polyNm', '당선횟수', '처리구분', '특별위원회',
       '환경노동위원회', '여성가족위원회', '법제사법위원회', '농림축산식품해양수산위원회', '기획재정위원회', '국방위원회',
       '과학기술정보방송통신위원회', '산업통상자원중소벤처기업위원회', '국토교통위원회', '행정안전위원회', '보건복지위원회',
       '문화체육관광위원회', '교육위원회', '외교통일위원회', '정무위원회', '국회운영위원회', 'num_seats',
       'summary_re', '국회', '정당/선거', '안보', '사법', '행정', '재정', '중소기업', '에너지',
       '부동산', '금융', '자동차', '건설/기계/조선', '유통/무역', 'IT', '농축산', '복지', '의료/보건',
       '도시/교통', '교육', '환경', '노동', '치안/안전', '가족', '여성', '예체능', '반대당발언횟수',
       '정보위원회', '입법형태_일부개정법률안', '입법형태_전부개정', '입법형태_제정', '선출형태_비례대표',
       '선출형태_지역구', 'party_0', 'party_1', 'party_2', 'party_3', 'party_4',
       'party_5', '정당다양성_0.0', '정당다양성_1.0', '상임위 상정 여부_0.0', '상임위 상정 여부_1.0'],
      dtype='object')

In [4]:
#소관위에서 대표발의자와 다른 정당 의원이 발언한 횟수 변수화하기

hearings = pd.read_excel('hearings_count.xlsx') #회의록 데이터
hearings = hearings.rename(columns={"법안번호": "법안코드"})
hearings = hearings.loc[ hearings['법안코드'].isin(df['법안코드']) ].reset_index().drop(['index'],axis=1)
hearings_df = pd.merge(hearings, df, how='left', on='법안코드')

In [5]:
from datetime import datetime

party_21th = pd.read_csv('party_21th.csv')
party_21th = party_21th[['emp_nm', 'poly_nm']]
party_21th = party_21th.rename(columns={'emp_nm':'화자'})
df_merged = pd.merge(hearings_df, party_21th, how='left', on='화자')
#df_merged = df_merged.rename(columns={'정당':'제안자 정당'}).reset_index()
df_merged = df_merged.rename(columns={'poly_nm':'화자 정당'})

In [8]:
party_21th = party_21th.rename(columns={'화자':'제안자'})
df_merged = pd.merge(df_merged, party_21th, how='left', on='제안자')
df_merged = df_merged.rename(columns={'poly_nm':'제안자 정당'})

In [10]:
df_merged['상임위원장 정당'] = '더불어민주당'

for i in range(len(df_merged)):
    if df_merged.loc[i, '제안자'] == '위원장': #위원장이 발의한 경우 상임위원장 정당을 제안자 정당으로 본다
        df_merged.loc[i, '제안자 정당'] = df_merged.loc[i, '상임위원장 정당']
        
for i in range(len(df_merged)):
    if df_merged.loc[i, '제안자'] == '정부':   #정부가 발의한 경우 더불어민주당(20대 국회 하반기: 문재인 정부)을 제안자 정당으로 본다
        df_merged.loc[i, '제안자 정당'] = '더불어민주당'

In [16]:
bill_no = list(set(df_merged['법안코드']))

cnt_list = []

for no in bill_no:
    bill_df = df_merged[df_merged['법안코드']==no].reset_index()
    cnt = 0
    for i in range(len(bill_df)):
        if bill_df['제안자 정당'][0] != bill_df.loc[i, '화자 정당'] and type(bill_df.loc[i, '화자 정당']) == str:
            cnt+=bill_df.loc[i, '횟수']
    cnt_list.append(cnt)

bill_cnt = pd.DataFrame({'반대당발언횟수': cnt_list, '법안코드':bill_no})

In [24]:
data = [tuple(x) for x in bill_cnt.to_numpy()]

sql = 'UPDATE bills SET 반대당발언횟수=반대당발언횟수+? WHERE 법안코드=?'
cursor.executemany(sql, data)
conn.commit()
conn.close()

In [37]:
bill_df = pd.merge(df, bill_cnt, how='left', on='법안코드')

In [38]:
bill_df.to_csv('21th_bill_preprocessed.csv', encoding='utf-8-sig') #csv파일로 저장

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("bills.db")
df = pd.read_sql("select * from bills_2021 where proposeDt >= '2021-01-01'", con = conn)

In [3]:
df.columns

Index(['level_0', 'billId', 'billName', 'billNo', 'passGubn', 'procStageCd',
       'proposeDt', 'proposerKind', 'summary', 'generalResult', 'procDt',
       'index', '공동발의자수', '공동발의평균선수', 'diversity', 'presentDt', 'COMMITTEE',
       'PROC_RESULT', 'RST_PROPOSER', 'PUBL_PROPOSER', 'COMMITTEE_ID',
       'polyNm', '당선횟수', '선출형태'],
      dtype='object')

In [4]:
df[['billId', 'billName', 'presentDt']]

,billId,billName,presentDt
0,PRC_G2K1E0H1K0X5L1V1X1O3E1K0X3W7Z0,전통시장 및 상점가 육성을 위한 특별법 일부개정법률안(이동주의원등18인),None
1,PRC_V2H1J0K1U0M4V1W8A2A4F1Y5A8D3E0,코로나19 감염병 피해 소상공인등 구제에 관한 특별법안(이동주의원등22인),None
2,PRC_L2O1Q0C1L0F4F1R5O2O6T1N1S9D7A6,소상공인기본법 일부개정법률안(이동주의원등17인),None
3,PRC_F2O1J0G1D1B1I1K7K5D0A3K3Q6R8G4,입양특례법 일부개정법률안(양경숙의원 등 10인),None
4,PRC_W2P1T0Y1F1L1W1J7C4Y9G4R0W3Q1A4,아동복지법 일부개정법률안(양경숙의원 등 10인),None
...,...,...,...
180,PRC_D2P1I0I1I0B4V1E1N5S4O3P2E0D2T8,북한이탈주민의 보호 및 정착지원에 관한 법률 일부개정법률안(지성호의원 등 10인),None
181,PRC_J2Z1V0Q1W0G4C1U1K2A8T2H9P8Y5R2,기부금품의 모집 및 사용에 관한 법률 일부개정법률안(한정애의원 등 14인),None
182,PRC_B2F1X0I1H0L4B1Q1R2W0E5T6K6A8R0,공직선거법 일부개정법률안(고용진의원 등 14인),None
183,PRC_B2O0G1W1G2U6V1M2J5E4Z3N4C4B2P0,주민등록법 일부개정법률안(이영의원등11인),None


In [6]:
set(df['presentDt'])

{'2020-12-03', '2021-01-08', None, 'None'}

In [11]:
for i in range(len(df)):
    if df.loc[i, 'presentDt'] != None or df.loc[i, 'presentDt'] != 'None':
        print(df.loc[i, 'billId'] + ":" + df.loc[i, 'billName']  type(df.loc[i, 'presentDt'])) 

TypeError: can only concatenate str (not "type") to str

In [50]:
committee_set.remove(None)

In [49]:
committee_set = list(set(df['COMMITTEE']))
committee_set

['기획재정위원회',
 '보건복지위원회',
 '환경노동위원회',
 '행정안전위원회',
 None,
 '국토교통위원회',
 '농림축산식품해양수산위원회',
 '교육위원회',
 '정무위원회',
 '국회운영위원회',
 '법제사법위원회',
 '국방위원회',
 '문화체육관광위원회',
 '외교통일위원회',
 '산업통상자원중소벤처기업위원회',
 '과학기술정보방송통신위원회',
 '여성가족위원회']

In [51]:
committee_set

['기획재정위원회',
 '보건복지위원회',
 '환경노동위원회',
 '행정안전위원회',
 '국토교통위원회',
 '농림축산식품해양수산위원회',
 '교육위원회',
 '정무위원회',
 '국회운영위원회',
 '법제사법위원회',
 '국방위원회',
 '문화체육관광위원회',
 '외교통일위원회',
 '산업통상자원중소벤처기업위원회',
 '과학기술정보방송통신위원회',
 '여성가족위원회']

In [45]:
set(df['proposeDt'])

{'2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-11',
 '2021-01-12'}

In [6]:
set(df['procStageCd'])

{'공포',
 '대안반영폐기',
 '본회의부의안건',
 '본회의불부의',
 '본회의의결',
 '소관위심사',
 '소관위심사보고',
 '소관위접수',
 '접수',
 '정부이송',
 '철회',
 '체계자구심사',
 '폐기'}

In [8]:
set(df['generalResult'])

{None, '대안반영폐기', '부결', '수정가결', '원안가결', '철회', '폐기'}

In [24]:
type(df.loc[4000, 'presentDt'])

str

In [46]:
df.columns

Index(['index', 'billId', 'billName', 'billNo', 'passGubn', 'procStageCd',
       'proposeDt', 'proposerKind', 'summary', 'generalResult', 'procDt',
       '공동발의자수', 'diversity', 'presentDt', 'COMMITTEE', 'PROC_RESULT',
       'RST_PROPOSER', 'PUBL_PROPOSER', 'COMMITTEE_ID', 'polyNm'],
      dtype='object')

In [2]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("bills_preprocessed.db")
df = pd.read_sql("select * from bills", con = conn)

In [5]:
df.columns

Index(['level_0', '법안코드', '법안명', 'billNo', 'passGubn', 'procStageCd', '접수일자',
       'proposerKind', 'summary', 'generalResult', 'procDt', 'index', '공동발의자수',
       '공동발의평균선수', '정당다양성', 'presentDt', 'COMMITTEE', 'PROC_RESULT', '제안자',
       'PUBL_PROPOSER', 'COMMITTEE_ID', 'polyNm', '당선횟수', '선출형태', '처리구분',
       '특별위원회', '환경노동위원회', '여성가족위원회', '법제사법위원회', '농림축산식품해양수산위원회', '기획재정위원회',
       '국방위원회', '과학기술정보방송통신위원회', '산업통상자원중소벤처기업위원회', '국토교통위원회', '행정안전위원회',
       '보건복지위원회', '문화체육관광위원회', '교육위원회', '외교통일위원회', '정무위원회', '국회운영위원회', 'party',
       'num_seats', '입법형태', 'summary_re', '국회', '정당/선거', '안보', '사법', '행정',
       '재정', '중소기업', '에너지', '부동산', '금융', '자동차', '건설/기계/조선', '유통/무역', 'IT',
       '농축산', '복지', '의료/보건', '도시/교통', '교육', '환경', '노동', '치안/안전', '가족', '여성',
       '예체능', '상임위 상정 여부'],
      dtype='object')

In [4]:
set(df['proposerKind'])

{'기타', '위원장', '의원', '의장', '정부'}

In [ ]:
df[df['propose']]

In [ ]:
for i in range(len(df)):
    if df.loc[i, 'proposerKind'] == '위원장':
        df.loc[i, 'party'] = 4
    elif df.loc[i, 'proposerKind'] == '정부':
        df.loc[i, 'party'] = 5


In [ ]:
for i in range(len(df)):
    if df.loc[i, 'billName'].__contains__('전부개정법률안'):
        df.loc[i, '입법형태'] = '전부개정'
    elif df.loc[i, 'billName'].__contains__('일부개정법률안'):
        df.loc[i, '입법형태'] = '일부개정법률안'
    else:
        df.loc[i, '입법형태'] = '제정'

In [12]:
import requests
mykey = "f9191bcb5fc3472890a5e84347ae5ebb"
url3 = f"https://open.assembly.go.kr/portal/openapi/nwvrqwxyaytdsfvhu?KEY={mykey}&Type=json&pSize=300"
req = requests.get(url3).json()
congressman = pd.DataFrame(req['nwvrqwxyaytdsfvhu'][1]['row'])

for i in range(len(congressman)):
    congressman.loc[i, '당선횟수'] = len(congressman.loc[i, 'UNITS'].split(', '))

In [14]:
congressman['ELECT_GBN_NM']

0       지역구
1       지역구
2       지역구
3       지역구
4      비례대표
       ... 
295     지역구
296     지역구
297     지역구
298     지역구
299     지역구
Name: ELECT_GBN_NM, Length: 300, dtype: object